### Install dependancies

In [1]:
!pip install pillow pandas datasets
!pip install -U scikit-learn

Looking in indexes: https://nexus.iisys.de/repository/ki-awz-pypi-group/simple, https://pypi.org/simple
Looking in indexes: https://nexus.iisys.de/repository/ki-awz-pypi-group/simple, https://pypi.org/simple


### Import libs

In [2]:
import json
import os
from datasets import Dataset, Features, Image, Value, Sequence, Split
import pandas as pd
from sklearn.model_selection import train_test_split
from PIL import Image as PILImage
import numpy as np

### Render ASCII Board

In [3]:
def check_win(grid):
    grid = np.array(grid)

    for i in range(3):

        # Check each row
        if grid[i, 0] == grid[i, 1] == grid[i, 2] != 0:
            return grid[i, 0]

        # Check each column
        if grid[0, i] == grid[1, i] == grid[2, i] != 0:
            return grid[0, i]

    # Check both diagonals
    if (grid[0, 0] == grid[1, 1] == grid[2, 2] != 0) or (grid[2, 0] == grid[1, 1] == grid[0, 2] != 0):
        return grid[1, 1]

    # Draw
    if np.all(grid != 0):
        return -1

    # Ongoing
    return 0

def reconstruct_board_matrix(global_state_list):
    board_matrix = [[[[0 for _ in range(3)] for _ in range(3)] for _ in range(3)] for _ in range(3)]
    
    global_status = [[0 for _ in range(3)] for _ in range(3)]

    for cell in global_state_list:
        g_r, g_c = cell['global_row'], cell['global_col']
        l_r, l_c = cell['local_row'], cell['local_col']
        player = cell['player']
        
        if 0 <= g_r < 3 and 0 <= g_c < 3 and 0 <= l_r < 3 and 0 <= l_c < 3:
            board_matrix[g_r][g_c][l_r][l_c] = player

    for g_r in range(3):
        for g_c in range(3):
            local_board = board_matrix[g_r][g_c]
            status = check_win(local_board)
            global_status[g_r][g_c] = status
            
    return board_matrix, global_status

def get_unplayable_boards(global_status):
    # all boards that have been won (1 or 2) or tied (3).
    unplayable_boards = []
    for g_r in range(3):
        for g_c in range(3):
            status = global_status[g_r][g_c]
            if status != 0:
                unplayable_boards.append({
                    "global_row": g_r,
                    "global_col": g_c
                })
    return unplayable_boards

In [4]:
def render_ascii_board(global_state_list):
    """
    Renders 9 separate 3x3 grids with explicit 0-1-2 axis labels.
    Critical for small datasets to ground the coordinates visually.
    """
    symbols = {0: '.', 1: 'X', 2: 'O'}

    # Map (g_row, g_col, l_row, l_col) -> symbol
    state_map = {}
    for cell in global_state_list:
        key = (cell['global_row'], cell['global_col'], cell['local_row'], cell['local_col'])
        state_map[key] = symbols.get(cell['player'], '?')

    board_sections = []

    # Iterate through Global Boards
    for g_r in range(3):
        for g_c in range(3):
            # Header with Global Coordinates
            section = [f"=== Global Board [Row {g_r}, Col {g_c}] ==="]

            # Column Axis Header (indent to match cell spacing)
            section.append("    0 1 2")
            section.append("   -------")

            # Render rows with Row Axis Label
            for l_r in range(3):
                row_cells = []
                for l_c in range(3):
                    val = state_map.get((g_r, g_c, l_r, l_c), '.')
                    row_cells.append(val)
                # Format: "0 | . X O"
                section.append(f"{l_r} | " + " ".join(row_cells))

            board_sections.append("\n".join(section))

    return "\n\n".join(board_sections)

### Data Generator

In [5]:
def gen_data():
    from config import SYNTHETIC_LOG_FILE_PATH, DATASET_FOLDER, IMAGES_FOLDER
    
    if not os.path.exists(SYNTHETIC_LOG_FILE_PATH):
        print(f"Error: Input file '{SYNTHETIC_LOG_FILE_PATH}' not found.")
        print("Please ensure you are running this script from the 'ultimate-tic-tac-toe' folder.")
        return

    print(f"Processing {SYNTHETIC_LOG_FILE_PATH}...")

    with open(SYNTHETIC_LOG_FILE_PATH, 'r') as f:
        for line_num, line in enumerate(f):
            if not line.strip():
                continue

            try:
                entry = json.loads(line)
            except json.JSONDecodeError:
                print(f"Skipping invalid JSON at line {line_num}")
                continue

            full_img_path = entry.get("image path")
            if not os.path.exists(full_img_path):
                print(f"Warning: Image not found at {full_img_path}. Skipping.")
                continue

            try:
                img_obj = PILImage.open(full_img_path).convert("RGB")
            except Exception as e:
                print(f"Error loading image {full_img_path}: {e}")
                continue

            global_state_list = entry.get("global state", [])
            
            try:
                board_matrix, global_status = reconstruct_board_matrix(global_state_list)
                unplayable_boards = get_unplayable_boards(global_status)
            except Exception as e:
                print(f"Error calculating board status for {full_img_path}: {e}. Skipping.")
                continue
            
            ascii_board_str = render_ascii_board(global_state_list)
            
            yield {
                "image": img_obj,
                "player": entry.get("player"),
                "global state": global_state_list,
                "unplayable_boards": unplayable_boards,
                "ascii_board": ascii_board_str,
                "allowed_squares": entry.get("allowed squares"),
                "best_move": entry.get("best move"),
                "legal_moves": entry.get("legal moves", []),
                "chain_of_thought": entry.get("chain of thought", "")
            }

### Main Execution

In [6]:
def main():
    features = Features({
        "image": Image(),
        "player": Value("int32"),
        "ascii_board": Value("string"),
        "chain_of_thought": Value("string"),
        
        "best_move": {
            "global_row": Value("int32"),
            "global_col": Value("int32"),
            "local_row": Value("int32"),
            "local_col": Value("int32"),
        },
        
        "allowed_squares": Sequence(Value("int32")), 
        
        "unplayable_boards": [{
            "global_row": Value("int32"),
            "global_col": Value("int32"),
        }],

        "legal_moves": [{
            "global_row": Value("int32"),
            "global_col": Value("int32"),
            "local_row": Value("int32"),
            "local_col": Value("int32"),
        }],
        
        "global state": [{
            "global_row": Value("int32"),
            "global_col": Value("int32"),
            "local_row": Value("int32"),
            "local_col": Value("int32"),
            "player": Value("int32"),
        }]
    })

    ds = Dataset.from_generator(gen_data, features=features)

    if len(ds) == 0:
        print("No data loaded.")
        return

    print(f"Loaded {len(ds)} samples.")

    # 90% Train, 5% Test, 5% Val
    train_testvalid = ds.train_test_split(test_size=0.2, seed=42)
    test_eval = train_testvalid['test'].train_test_split(test_size=0.5, seed=42)

    final_dataset = {
        'train': train_testvalid['train'],
        'test': test_eval['train'],
        'evaluate': test_eval['test']
    }

    from config import DATASET_FOLDER
    os.makedirs(DATASET_FOLDER, exist_ok=True)

    for split_name, dataset in final_dataset.items():
        file_name = f"{DATASET_FOLDER}/{split_name}.parquet"
        print(f"Saving {split_name} split to {file_name}...")
        dataset.to_parquet(file_name)

    print("\nSuccess! Dataset created in:", DATASET_FOLDER)


if __name__ == "__main__":
    main()

Generating train split: 0 examples [00:00, ? examples/s]

Processing logs/bot_moves_synthetic.jsonl...
Loaded 4004 samples.
Saving train split to uttt_qwen_dataset/train.parquet...


Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Saving test split to uttt_qwen_dataset/test.parquet...


Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Saving evaluate split to uttt_qwen_dataset/evaluate.parquet...


Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]


Success! Dataset created in: uttt_qwen_dataset
